# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

**Reminder about our [Rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins) and the [Policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating)**

We start by loading spaCy:

In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [2]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(' '))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [3]:
import bz2
import csv
import pandas as pd

with bz2.open('ner-train.tsv.bz2', 'rt') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open('ner-dev.tsv.bz2', 'rt') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [4]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [5]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    # TODO: Replace the next line with your own code
    # Precision
    precision = (len(pred.intersection(gold)))/len(pred)
    print(precision)
    # recall
    recall = (len(pred.intersection(gold)))/len(gold)
    print(recall)
    # f1score
    f1score = (2*(precision*recall))/(precision+recall)
    print(f1score)
    pass

To test your code, you can run the following cell:

In [6]:
evaluation_report(set(range(3)), set(range(5)))

0.6
1.0
0.7499999999999999


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [7]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for i, row in enumerate(df.itertuples(), 1):
        yield row.sentence_id, row.beg, row.end

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [9]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))


5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [10]:
# TODO: Write code here to run and evaluate the spaCy NER on the development data
my_span = set()
for i, row in enumerate(df_dev.itertuples(), 1):
    doc = nlp(row.sentence)
    for ent in doc.ents:
        my_span.add((row.sentence_id, ent.start, ent.end))
evaluation_report(spans_dev_gold, my_span)

0.5213954072029113
0.702213959776914
0.5984444764511019


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Here is a function that prints the false positives as well as the false negatives spans for a data frame, given a reference set of gold-standard spans and a candidate set of predicted spans.

In [11]:
from collections import defaultdict

def error_report(df, spans_gold, spans_pred):
    false_pos = defaultdict(list)
    for s, b, e in spans_pred - spans_gold:
        false_pos[s].append((b, e))
    false_neg = defaultdict(list)
    for s, b, e in spans_gold - spans_pred:
        false_neg[s].append((b, e))
    for row in df.drop_duplicates('sentence_id').itertuples():
        if row.sentence_id in false_pos or row.sentence_id in false_neg:
            print('Sentence:', row.sentence)
            for b, e in false_pos[row.sentence_id]:
                print('  FP:', ' '.join(row.sentence.split()[b:e]))
            for b, e in false_neg[row.sentence_id]:
                print('  FN:', ' '.join(row.sentence.split()[b:e]))

Use this function to inspect and analyse the errors that the automated prediction makes. Can you see any patterns? Base your analysis on the first 500 rows of the training data. Summarize your observations in a short text.

In [12]:
# TODO: Write code here to do your analysis
error_report(df_dev,spans_dev_gold,my_span)

Sentence: CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
  FN: LEICESTERSHIRE
Sentence: LONDON 1996-08-30
  FP: 1996-08-30
Sentence: West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .
  FP: Friday
  FP: 39
  FP: four
  FP: 38
  FP: two days
Sentence: After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .
  FP: first
  FP: 94
  FP: 296
  FP: 83
  FP: 83
  FP: three
  FP: the opening morning
Sentence: Trailing by 213 , Somerset got a solid start to their second innings before Simmons stepped in to bundle them out for 174 .
  FP: 174
  FP: second
  FP: 213
Sentence: Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip

*TODO: Write a short text that summarises the errors that you observed*

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to post-process the output produced by spaCy. To filter out specific labels it is useful to know the named entity label scheme, which can be found in the [model's documentation](https://spacy.io/models/en#en_core_web_sm). You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [13]:
# TODO: Write code here to improve the span prediction from Problem 2
my_span2 = set()
for i, row in enumerate(df_dev.itertuples(), 1):
    doc = nlp(row.sentence)
    for ent in doc.ents:
            if ent.label_ in {"DATE","QUANTITY","TIME","MONEY","PERCENT", "CARDINAL"}:
                i = 1
            else:
                my_span2.add((row.sentence_id, ent.start, ent.end))
evaluation_report(spans_dev_gold, my_span2)

0.8126101429410613
0.701368936961298
0.7529027576197387


In [14]:
error_report(df_dev,spans_dev_gold,my_span2)

Sentence: CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
  FN: LEICESTERSHIRE
Sentence: After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .
  FP: first
Sentence: Trailing by 213 , Somerset got a solid start to their second innings before Simmons stepped in to bundle them out for 174 .
  FP: second
Sentence: Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip on their match against Yorkshire at Headingley .
  FP: Yorkshire at
  FN: Yorkshire
  FN: Essex
Sentence: Hussain , considered surplus to England 's one-day requirements , struck 158 , his first championship century of the season , as Essex reached 372 and took a first innings lead of 82 .
  FP: first
  FP: first
Sentence: By the close Yorkshire had turned that into a 37-run advantage but of

Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the *predicted* start and end positions for each token span, rather than the gold positions. As the `label` of each span, you can use the special value `--NME--`.

In [15]:
# TODO: Write code here to store the predicted spans in a new data frame
new_dict = {
    'sentence_id': [],
    'sentence': [],
    'beg': [],
    'end': [],
    'label': []
}
for tup in my_span2:
    new_dict['sentence_id'].append(tup[0])
    temp = df_dev.loc[df_dev['sentence_id'] == tup[0]]
    new_dict['sentence'].append(temp['sentence'].iloc[0])  
    new_dict['beg'].append(tup[1])
    new_dict['end'].append(tup[2])
    new_dict['label'].append('--NME--')
new_df = pd.DataFrame.from_dict(new_dict)

In [16]:
print(new_df)

     sentence_id                                           sentence  beg  end  \
0       0985-003  An eye specialist told the 35-year-old Bruno t...    6    7   
1       1066-041                              Watford 4 2 0 2 4 5 6    0    1   
2       1033-003  Flavio Cotti , the chairman of the Organisatio...   15   16   
3       1097-020  " It does n't look all that bad , " Edberg sai...   28   30   
4       1121-007  President Bill Clinton on Friday ordered the U...   17   18   
...          ...                                                ...  ...  ...   
5102    1011-012  Among those seen as having an interest in buyi...   18   20   
5103    0966-105                    7. Marc Blume ( Germany ) 10.48    4    5   
5104    1090-014  Seles 's sore left shoulder and a wrist injury...   13   16   
5105    1101-009  Jacquet , beginning the 22-month countdown to ...    0    1   
5106    1095-017             Los Angeles 7 PHILADELPHIA 6 ( in 12 )    0    2   

        label  
0     --NME

## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [17]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for i, row in enumerate(df.itertuples(), 1):
        yield row.sentence_id, row.beg, row.end, row.label

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

**Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.**

In [18]:
# TODO: Write code here to implement the baseline
labels_dev_gold = set(gold_mentions(df_dev))
print(len(labels_dev_gold))
baseline_labels = set()
for row in new_df.itertuples():
    sentence_li = row.sentence.split()
    templabel = ('_').join(sentence_li[row.beg:row.end])
    baseline_labels.add((row.sentence_id, row.beg, row.end, templabel))
    
print(len(baseline_labels))
evaluation_report(labels_dev_gold, baseline_labels)

5917
5107
0.3011552770706873
0.25992901808348823
0.27902757619738755


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on) as mentions of the entities (pages) that they link to. This allows us to harvest long lists of mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [19]:
with bz2.open('kb.tsv.bz2', 'rt') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [40]:
df_kb.loc[df_kb.mention == 'Lincoln']

,mention,entity,prob
10559,Lincoln,"Lincoln,_Nebraska",0.748718
10560,Lincoln,Abraham_Lincoln,0.220513
10561,Lincoln,Lincoln_City_F.C.,0.030769


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing come first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [21]:
# TODO: Write code here to implement the "most probable entity" method.
mostprob_labels = set()
for row in new_df.itertuples():
    sentence_li = row.sentence.split()
    templabel = ('_').join(sentence_li[row.beg:row.end])
    problabel = df_kb.loc[df_kb.mention == templabel]
    if problabel.empty:
        mostprob_labels.add((row.sentence_id, row.beg, row.end, '--NME--'))
    else:
        mostprob_labels.add((row.sentence_id, row.beg, row.end, problabel['entity'].iloc[0]))
    
print(len(mostprob_labels))
evaluation_report(labels_dev_gold, mostprob_labels)

5107
0.4689641668298414
0.4047659286800744
0.43450653120464444


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier that is trained to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [22]:
with bz2.open('contexts.tsv.bz2') as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [23]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in @ and the second in 1983
1,1970,FIFA_World_Cup,America 1975 and during the @ and 1978 World C...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the @
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the @ and ...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the @ The present capa...


Note that, in each context, the position of the mention is indicated by the `@` symbol.

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [25]:
df_contexts[df_contexts.mention == 'Lincoln']

,mention,entity,context
41465,Lincoln,"Lincoln,_Nebraska",Nebraska Concealed Handgun Permit In @ municip...
41466,Lincoln,"Lincoln,_Nebraska",Lazlo restaurants are located in @ and Omaha C...
41467,Lincoln,"Lincoln,_Nebraska",California Washington Overland Park Kansas @ N...
41468,Lincoln,"Lincoln,_Nebraska",City Missouri Omaha Nebraska and @ Nebraska It...
41469,Lincoln,"Lincoln,_Nebraska",by Sandhills Publishing Company in @ Nebraska USA
...,...,...,...
41609,Lincoln,Lincoln_City_F.C.,@ Leyton Orient
41610,Lincoln,Lincoln_City_F.C.,English division three Swansea @
41611,Lincoln,Lincoln_City_F.C.,league membership narrowly edging out @ on goa...
41612,Lincoln,Lincoln_City_F.C.,@ Cambridge


Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [54]:
# TODO: Write code here to implement the context-sensitive disambiguation method
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

mentions = df_contexts['mention'].unique()
print(mentions)
naive_dict = dict()
for mention in mentions:
    #if df_kb.loc[df_kb.mention == mention].shape[0] > 1:

    priorsm = []
    for row in df_kb.loc[df_kb.mention == mention].itertuples():
        priorsm.append(row.prob)
    
    pipe = Pipeline([('scaler', CountVectorizer()),('clf', MultinomialNB(class_prior=priorsm))])
    trainX, trainY = df_contexts[df_contexts.mention == mention]['context'], df_contexts[df_contexts.mention == mention]['entity']
    pipe.fit(trainX, trainY)
    naive_dict[mention] = pipe
   
   
print(naive_dict)    

['1970' '1990 World Cup' '1992' ... 'wartime' 'west' 'western']
{'1970': Pipeline(steps=[('scaler', CountVectorizer()),
                ('clf', MultinomialNB(class_prior=[0.5, 0.5]))]), '1990 World Cup': Pipeline(steps=[('scaler', CountVectorizer()),
                ('clf',
                 MultinomialNB(class_prior=[0.9791666666666666,
                                            0.0208333333333333]))]), '1992': Pipeline(steps=[('scaler', CountVectorizer()),
                ('clf',
                 MultinomialNB(class_prior=[0.9857142857142858,
                                            0.0142857142857142]))]), '1996': Pipeline(steps=[('scaler', CountVectorizer()),
                ('clf',
                 MultinomialNB(class_prior=[0.9410029498525072,
                                            0.0560471976401179,
                                            0.0029498525073746]))]), '2000 Summer Olympics': Pipeline(steps=[('scaler', CountVectorizer()),
                ('clf',
         

In [76]:
context_labels = set()
for row in new_df.itertuples():
    sentence_li = row.sentence.split()
    templabel = ('_').join(sentence_li[row.beg:row.end])
    problabel = df_kb.loc[df_kb.mention == templabel]
    if templabel in naive_dict:
        clflabel = naive_dict[templabel].predict([row.sentence])
        context_labels.add((row.sentence_id, row.beg, row.end, clflabel[0]))
    elif not problabel.empty:  
        context_labels.add((row.sentence_id, row.beg, row.end, problabel['entity'].iloc[0]))
    else:
        context_labels.add((row.sentence_id, row.beg, row.end, '--NME--')) 
print(len(context_labels))
evaluation_report(labels_dev_gold, context_labels)

5107
0.42725670648130015
0.3687679567348318
0.39586357039187225


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1.

## Reflection questions

The following reflection questions will help you prepare for the diagnostic test. Answer each of them in the form of a short text and put your answers in the cell below. You will get feedback on your answers from your lab assistant.

**RQ 5.1:** In Problem&nbsp;3, you did an error analysis on the task of recognizing text spans mentioning named entities. Summarize your results. Pick one type of error that you observed. How could you improve the model&rsquo;s performance on this type of error? What resources (such as domain knowledge, data, compute) would you need to implement this improvement?

**RQ 5.2:** Thinking back about Problem&nbsp;6, explain what the word *context* refers to in the task addressed there, and how context can help to disambiguate between different entities. Suggest other types of context that you could use for disambiguation.

**RQ 5.3:** One type of entity mentions that we did not cover explicitly in this lab are pronouns. As an example, consider the sentence pair *Ruth Bader Ginsburg was an American jurist*. *She served as an associate justice of the Supreme Court from 1993 until her death in 2020*. What facts would you want to extract from this sentence pair? How do pronouns make fact extraction hard?

*TODO: Enter your answers here*

**This was the last lab in the Text Mining course. Congratulations! 🥳**